In [18]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-07-12 20:11:31--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-12 20:11:31 (44.9 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [19]:
import minsearch

In [20]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-07-12 20:11:32--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.2’

documents.json.2    100%[===================>] 642.90K  --.-KB/s    in 0.009s  

2024-07-12 20:11:32 (67.5 MB/s) - ‘documents.json.2’ saved [658332/658332]



In [21]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/parse-faq.ipynb

--2024-07-12 20:11:32--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/parse-faq.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5860 (5.7K) [text/plain]
Saving to: ‘parse-faq.ipynb.2’

parse-faq.ipynb.2   100%[===================>]   5.72K  --.-KB/s    in 0s      

2024-07-12 20:11:32 (27.0 MB/s) - ‘parse-faq.ipynb.2’ saved [5860/5860]



In [22]:
import json

In [23]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [24]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[10]

{'text': 'It depends on your background and previous experience with modules. It is expected to require about 5 - 15 hours per week. [source1] [source2]\nYou can also calculate it yourself using this data and then update this answer.',
 'section': 'General course-related questions',
 'question': 'Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
index = minsearch.Index(text_fields=["question","text","section"],keyword_fields=["course"])

In [26]:
q = 'the course has already started, can I still join?'

In [27]:
index.fit(documents)

In [28]:
boost = {'question':3.0, 'section':0.5}

index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course':'data-engineering-zoomcamp'},
    num_results=3
)
    

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [29]:
from openai import OpenAI

In [30]:
client = OpenAI()

In [33]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role" : "user", "content" : q}]
)

In [34]:
response

ChatCompletion(id='chatcmpl-9kGvcqk4XTozmxIqBaCADJYyt1yZk', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="It's best to check with the course provider or instructor directly to see if you can still join. Some courses have flexible enrollment periods or late registration options, while others might have stricter deadlines. If it's an online course, there might be more flexibility compared to an in-person class. Additionally, you may need to catch up on any material you missed, so be sure to ask about that as well.", role='assistant', function_call=None, tool_calls=None))], created=1720815188, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d33f7b429e', usage=CompletionUsage(completion_tokens=80, prompt_tokens=18, total_tokens=98))

In [35]:
response.choices[0].message.content

"It's best to check with the course provider or instructor directly to see if you can still join. Some courses have flexible enrollment periods or late registration options, while others might have stricter deadlines. If it's an online course, there might be more flexibility compared to an in-person class. Additionally, you may need to catch up on any material you missed, so be sure to ask about that as well."

In [42]:
prompt_template = """
You'are a course teaching assistant for Machine Learning. Answer the question and explain the reasoning behind your answer. Use the context provided to answer that question.
Use only the facts provided in the context. If there is no answer, return a generic answer.
Be polite and friendly.

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [54]:
context = ""
n = 0

for doc in documents:
    if n < 51:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        n = n + 1
    

In [55]:
context

'section: General course-related questions\nquestion: Course - When will the course start?\nanswer: The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours\'\' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club\'s Slack and join the channel.\n\nsection: General course-related questions\nquestion: Course - What are the prerequisites for this course?\nanswer: GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites\n\nsection: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\nBe aware, however, that there will be deadli

In [56]:
prompt = prompt_template.format(question=q, context=context).strip()

In [57]:
print(prompt)

You'are a course teaching assistant for Machine Learning. Answer the question and explain the reasoning behind your answer. Use the context provided to answer that question.
Use only the facts provided in the context. If there is no answer, return a generic answer.
Be polite and friendly.

QUESTION: the course has already started, can I still join?

CONTEXT: section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

section: General course-related questions
question: Course - What are the prerequ

In [59]:
responseLLM = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role" : "user", "content" : prompt}]
)

In [60]:
responseLLM.choices[0].message.content

"Yes, you can still join the course even after the start date. According to the provided context, even if you don't register, you are still eligible to submit the homeworks. However, please be aware that there will be deadlines for turning in the final projects, so it's advisable not to leave everything for the last minute.\n\nIf you need any assistance or have further questions, feel free to reach out. Welcome to the course! 😊"